# Load data

In [1]:
# !pip install --upgrade pip
# !pip install apache_beam
# !pip install datasets

# from datasets import load_dataset

# dataset = load_dataset("wiki40b", "en")
# train_dataset = dataset['train']
# validation_dataset = dataset['validation']
# test_dataset = dataset['test']

  Using cached apache_beam-2.48.0-cp310-cp310-win_amd64.whl (4.7 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.48.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


  Using cached datasets-2.13.0-py3-none-any.whl (485 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1


Found cached dataset wiki40b (C:/Users/14743/.cache/huggingface/datasets/wiki40b/en/1.1.0/d15702fbf830e65fd775c50946364ff0c02fd3089b31887fabf97c2dad970760)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
# train_dataset

Dataset({
    features: ['wikidata_id', 'text', 'version_id'],
    num_rows: 2926536
})

In [97]:
# train_dataset.save_to_disk("train_data")
# validation_dataset.save_to_disk("validation_data")
# test_dataset.save_to_disk("test_data")

Saving the dataset (0/19 shards):   0%|          | 0/2926536 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/163597 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/162274 [00:00<?, ? examples/s]

In [102]:
# from datasets import load_from_disk
# load_train = load_from_disk("train_data")
# load_test = load_from_disk("test_data")
# load_train

Dataset({
    features: ['wikidata_id', 'text', 'version_id'],
    num_rows: 2926536
})

# Module import

In [1]:
import numpy as np
import torch
import torch.nn as nn

import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from random import choice

import random
import string
import math

import numpy as np

from transformers import BertTokenizer, BartTokenizer, BartModel, BartForConditionalGeneration, AutoTokenizer, Text2TextGenerationPipeline, pipeline
from transformers import TrainingArguments, Trainer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, load_dataset, load_from_disk

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\14743\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
torch.cuda.is_available()

True

In [3]:
device_id= 0 if torch.cuda.is_available() else 'cpu'
# device = torch.device(device_id)
device = torch.device('cpu')
device

device(type='cpu')

In [4]:
train_dataset = load_from_disk("train_data")
validation_dataset = load_from_disk("validation_data")
test_dataset = load_from_disk("test_data")

In [5]:
validation_dataset

Dataset({
    features: ['wikidata_id', 'text', 'version_id'],
    num_rows: 163597
})

## Define Corruption

In [19]:
def add_spaces(sentence):
    org_num_space = sentence.count(" ")
    num_adds = math.floor(len(sentence) // 5) + 1
    positions_to_add = random.sample(range(len(sentence)), num_adds)

    for pos in positions_to_add[::-1]:
        sentence = sentence[:pos] + ' ' + sentence[pos:]

    return sentence

def delete_spaces(sentence):
    chars = list(sentence)

    org_num_space = sentence.count(" ")
    # num_deletions = math.floor(len(sentence) // 5) + 1
    positions = [i for i, char in enumerate(chars) if char == ' ']
    num_deletions = math.floor(len(positions) // 2)
    spaces_to_delete = random.sample(positions, num_deletions)

    for index in spaces_to_delete:
        chars[index] = ''

    return ''.join(chars)

def delete_characters(sentence):
    chars = list(sentence)

    # num_deletions = math.floor(len(sentence) // 5) + 1
    positions = [i for i, char in enumerate(chars) if char != ' ']
    num_deletions = math.floor(len(positions) // 2) 
    chars_to_delete = random.sample(positions, num_deletions)

    for index in chars_to_delete:
        chars[index] = ''
    return ''.join(chars)

def delete_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation))

def change_capitalization(sentence):
    return ''.join(random.choice((str.upper, str.lower))(char) for char in sentence)

In [7]:

# weights = np.random.uniform(size=4)
# print(weights)

[0.49988349 0.39295922 0.71062144 0.87478552]


## Data preprocessing

In [20]:
# print(train_dataset)

def perform_corruption(dataset, num_samples):
    original_sentences = []
    corrupted_sentences = []

    for i in range(num_samples): # change number of wiki
        text = dataset[i]['text']
        # print(text)
        # print("\n\n")

        # Extract each wiki by paragraphs
        lines = text.split("\n")

        paragraphs = []
        for index, line in enumerate(lines):
            # print(line)
            # print(index)
            if line == "_START_PARAGRAPH_":
                para = lines[index+1]
                para_list = para.split("_NEWLINE_")
                paragraphs += para_list
        # print(paragraphs)


        # Extract each paragraphs by sentence
        sentences = []
        for sentence in paragraphs:
            sentences += sent_tokenize(sentence)
        # print(sentences)
        original_sentences.extend(sentences)


        # Add Corruption
        for sentence in sentences:
            weights = np.random.uniform(size=4)
            corrupted_s = sentence

            # Add/Delete spaces
            if weights[0] > 0.7:
                corrupted_s = add_spaces(corrupted_s)
            # else:
            if weights[0] > 0.7:
                corrupted_s = delete_spaces(corrupted_s)
            # Delete characters
            if weights[1] > 0.7:
                corrupted_s = delete_characters(corrupted_s)
            # Delete punctuation
            if weights[2] > 0.7:
                corrupted_s = delete_punctuation(corrupted_s)
            # Change capitalization
            if weights[3] > 0.7:
                corrupted_s = change_capitalization(corrupted_s)

            corrupted_sentences.append(corrupted_s)

    return original_sentences, corrupted_sentences

train_original_sentences, train_corrupted_sentences = perform_corruption(train_dataset, 250)
val_original_sentences, val_corrupted_sentences = perform_corruption(validation_dataset, 20)

# print("original: ", original_sentences)
print(len(train_original_sentences))
print(len(train_corrupted_sentences))
print(len(val_original_sentences))
# print("corrupted: ", corrupted_sentences)
print(len(val_corrupted_sentences))


4188
4188
395
395


In [21]:
with open("train_original.txt", 'w') as f:
    for sen in train_original_sentences:
        f.write(sen)
        f.write("\n")
    
with open("train_corrupted.txt", 'w') as f:
    for sen in train_corrupted_sentences:
        f.write(sen)
        f.write("\n")

with open("val_original.txt", 'w') as f:
    for sen in val_original_sentences:
        f.write(sen)
        f.write("\n")
    
with open("val_corrupted.txt", 'w') as f:
    for sen in val_corrupted_sentences:
        f.write(sen)
        f.write("\n")

In [14]:


# X = corrupted_sentences

# # Splitting the data into train, test, and validation sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

# # Printing the resulting sets
# print("Train data:", train_data)
# print("Test data:", test_data)
# print("Validation data:", val_data)


## Models

# BART

### Toy example of BART

In [15]:
"cuda:0" if torch.cuda.is_available() else "cpu"

'cuda:0'

In [16]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model.to(device)
# model.config

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [17]:
# def bart_correct(tokenizer, model, text: str, max_length: int = 128):

#     import numpy as np
#     inputs = tokenizer.encode(text, padding=True, max_length=32, truncation=True,
#                                 return_tensors='pt')
#     model.eval()
#     with torch.no_grad():
#         res = model(inputs).logits
#         res = np.argmax(res[0],axis=1)
#         res = res[1:-1]
#         decode_tokens = tokenizer.decode(res,skip_special_tokens=True).replace(' ', '')
#     return decode_tokens

In [18]:
# bart_correct(tokenizer, model, "I am [] happy.")

Ġ refers to spaces in the byte-level BART

## Load the data

In [19]:
def original_target_combine(original_file, target_file):
    original_lst = []
    target_lst = []

    with open(original_file, 'r') as f:
        for line in f:
            original_lst.append(line[:-1])
    with open(target_file, 'r') as f:
        for line in f:
            target_lst.append(line[:-1])

    results = []
    for i in range(len(original_lst)):
        results.append(original_lst[i] + '\t' + target_lst[i])

    return {"text": results}, original_lst, target_lst

In [20]:
train_combined_sentences, train_originals, train_targets = original_target_combine("train_original.txt", "train_corrupted.txt")
print(train_combined_sentences["text"][0])
print(train_originals[0])
print(train_targets[0])
train_combined_training_dataset = Dataset.from_dict(train_combined_sentences, split="train")
print(len(train_combined_sentences["text"]))

She started singing as a toddler, considering Márta Sebestyén a role model.	She started singing as a toddler considering Márta Sebestyén a role model
She started singing as a toddler, considering Márta Sebestyén a role model.
She started singing as a toddler considering Márta Sebestyén a role model
1367


In [21]:
val_combined_sentences, val_originals, val_targets = original_target_combine("val_original.txt", "val_corrupted.txt")
print(val_combined_sentences["text"][0])
print(val_originals[0])
print(val_targets[0])
val_combined_training_dataset = Dataset.from_dict(val_combined_sentences, split="train")
print(len(val_combined_sentences["text"]))

In the late 19th century and early 20th century, Ðelekovec was part of the Varaždin County of the Kingdom of Croatia-Slavonia.	In the late 19th century and early 20th century Ðelekovec was part of the Varaždin County of the Kingdom of CroatiaSlavonia
In the late 19th century and early 20th century, Ðelekovec was part of the Varaždin County of the Kingdom of Croatia-Slavonia.
In the late 19th century and early 20th century Ðelekovec was part of the Varaždin County of the Kingdom of CroatiaSlavonia
175


In [22]:
def tokenize_dataset(tokenizer, dataset, max_length_padding):    
    def tokenize_func(batch_data):
        corrupted_lst = []
        original_lst = []   
        for sentence in batch_data["text"]:
            original, corrput = sentence.split('\t', 1)
            corrupted_lst.append(corrput)
            original_lst.append(original)

        input_encodings = tokenizer.batch_encode_plus(
            corrupted_lst,              # List of source sequences
            padding="max_length",
            truncation=True,
            max_length=max_length_padding,
            return_tensors="pt"         # Return PyTorch tensors
        )

        target_encodings = tokenizer.batch_encode_plus(
            original_lst,  # List of source sequences
            padding="max_length",
            truncation=True,
            max_length=max_length_padding,
            return_tensors="pt"  # Return PyTorch tensors
        )

        return {
                "input_ids": input_encodings['input_ids'],
                "attention_mask": input_encodings['attention_mask'],
                "target_ids": target_encodings['input_ids'],
                "target_attention_mask": target_encodings['attention_mask']
            }
    
    dataset = dataset.map(tokenize_func, batched=True)
    columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
    dataset = dataset.with_format(type="torch", columns=columns)
    dataset = dataset.rename_column("target_ids", "labels")
    dataset = dataset.rename_column("target_attention_mask", "decoder_attention_mask")
    dataset = dataset.remove_columns("text")
    return dataset

In [23]:
max_length_padding = 128

In [24]:
final_training_data = tokenize_dataset(tokenizer, train_combined_training_dataset, max_length_padding)
final_val_data = tokenize_dataset(tokenizer, val_combined_training_dataset, max_length_padding)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

In [25]:
training_arguments = Seq2SeqTrainingArguments(
    output_dir="./BART_toy_results",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-3,
    save_steps=False,
    logging_steps=2
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_arguments,
    train_dataset=final_training_data,
    eval_dataset=final_val_data
)

trainer.train()
model.save_pretrained("./BART_toy_model")

c:\Users\14743\anaconda3\envs\nlp\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/860 [00:00<?, ?it/s]

{'loss': 12.7659, 'learning_rate': 0.0009976744186046512, 'epoch': 0.02}
{'loss': 11.4983, 'learning_rate': 0.0009953488372093024, 'epoch': 0.05}
{'loss': 7.7835, 'learning_rate': 0.0009930232558139536, 'epoch': 0.07}
{'loss': 4.0488, 'learning_rate': 0.0009906976744186047, 'epoch': 0.09}
{'loss': 3.1738, 'learning_rate': 0.0009883720930232557, 'epoch': 0.12}
{'loss': 2.5955, 'learning_rate': 0.0009860465116279071, 'epoch': 0.14}
{'loss': 2.8827, 'learning_rate': 0.000983720930232558, 'epoch': 0.16}
{'loss': 2.6524, 'learning_rate': 0.0009813953488372093, 'epoch': 0.19}
{'loss': 2.132, 'learning_rate': 0.0009790697674418604, 'epoch': 0.21}
{'loss': 2.416, 'learning_rate': 0.0009767441860465116, 'epoch': 0.23}
{'loss': 2.3847, 'learning_rate': 0.0009744186046511628, 'epoch': 0.26}
{'loss': 2.873, 'learning_rate': 0.000972093023255814, 'epoch': 0.28}
{'loss': 2.7776, 'learning_rate': 0.0009697674418604652, 'epoch': 0.3}
{'loss': 2.4881, 'learning_rate': 0.0009674418604651162, 'epoch': 0.

In [42]:
def model_decode(tokenizer, model, input, max_length_padding):
    inputs = tokenizer.batch_encode_plus([input], padding=True, 
                                         max_length=max_length_padding, 
                                         truncation=True, 
                                         return_tensors="pt")["input_ids"].cuda()

    model.eval()
    with torch.no_grad():
        print(inputs)
        output = model.generate(inputs, num_beams=2, length_penalty=1.0, max_length=50, min_length=0, no_repeat_ngram_size=3)
        # print(output)
        output_text = tokenizer.decode(output[0], skip_special_tokens=True)

        return output_text
    

model_decode(tokenizer, model, "The music s like wate", max_length_padding)

tensor([[  0, 133, 930, 579, 101, 885, 877,   2]], device='cuda:0')


'TheInOnAItThis theHe wasTheyAfterAs,At'

In [ ]:
# original_lst = []
# corrupted_lst = []

# with open("train_original.txt", 'r') as f:
#     for line in f:
#         original_lst.append(line[:-1])
# with open("train_corrupted.txt", 'r') as f:
#     for line in f:
#         corrupted_lst.append(line[:-1])

# training_dict = {"text": []}
# for i in range(len(original_lst)):
#     training_dict["text"].append(original_lst[i] + "\t" + corrupted_lst[i])

# toy_dataset = Dataset.from_dict(original_dict)
# print(training_dict["text"][0])
# print(original_lst[0])
# print(corrupted_lst[0])

In [ ]:
def tokenize_func(tokenizer, max_length_padding, original_lst, corrupted_lst):
    input_encodings = tokenizer.batch_encode_plus(
        corrupted_lst,              # List of source sequences
        padding="max_length",
        truncation=True,
        max_length=max_length_padding,
        return_tensors="pt"         # Return PyTorch tensors
    )

    target_encodings = tokenizer.batch_encode_plus(
        original_lst,  # List of source sequences
        padding="max_length",
        truncation=True,
        max_length=max_length_padding,
        return_tensors="pt"  # Return PyTorch tensors
    )

    return {
            "input_ids": input_encodings['input_ids'],
            "attention_mask": input_encodings['attention_mask'],
            "target_ids": target_encodings['input_ids'],
            "target_attention_mask": target_encodings['attention_mask']
        }

In [ ]:
tokenize_func(tokenizer, max_length_padding, original_lst, corrupted_lst)

NameError: name 'original_lst' is not defined

In [ ]:
def tokenize_dataset(tokenizer, dataset, max_length_padding)

In [ ]:
inputs = tokenizer.batch_encode_plus(
    original_lst,  # List of source sequences
    padding=True,
    truncation=True,
    return_tensors="pt"  # Return PyTorch tensors
)
inputs

{'input_ids': tensor([[    0,  2515,   554,  6970,    25,    10, 16290,     6,  2811,   256,
          1526,   338,  4349, 36357, 27668, 10598,    10,   774,  1421,     4,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 13584,  4388,  3618,    16,  2065, 13134,   930,   131,    79,
            78,   351,  4972,    13,  6970,    19, 30492, 32059,  3671,    29,
            11,    10,  2065, 13134,  2496,     6,     8,   163,  4891,  1021,
         13868,   448,     6,    10, 28093,   260, 18124, 23566, 20535,  1971,
             4,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,  3908,   209,  1177, 26660,  1634,    79, 20864,   198,     5,
           232,    31,     5,  5817,  9529,  3502,     6,   149,  457

In [ ]:



inputs = tokenizer(["Hello, my  dog is cute", "hooo hjnv hahseexa"], padding=True, truncation=True, return_tensors="pt")
outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
print(inputs)
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
tokenizer("Hello, my  dog is cute", "ho my god")
print(outputs.last_hidden_state.shape)
outputs

tokenizer.pad_token_id

{'input_ids': tensor([[    0, 31414,     6,   127,  1437,  2335,    16, 11962,     2,     1,
             1,     1],
        [    0,   298, 41242,  1368,   267, 37594,  2489,   298,  1090,  3463,
           102,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([2, 12, 768])


1

### BERT + Transformer

### GPT

## Evaluation